This Notebook will group stocks in sectors, find top n stocks of resptive sector based on user criteria, then will run a correlation, and find most divesrivfiable stocks (stocks with lowest correlation)

In [1]:
import time
import os 

start_time = time.time() # measure run time

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')

from utils.finance_utils import *
from utils.helpers import separate_corr_pairs
from utils.config import TOP_N_STOCKS

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py

---------------------------------
finance_utils.py successfully loaded, updated last Feb. 04 2025 1:01
---------------------------------



---------------------------------
helpers.py successfully loaded, updated last Feb. 04 2025
---------------------------------


Updated on 02/05/2025 12:55


Sector Valuation: Group by sectors, and find best stock in each sector

In [2]:
filtered_valuation_df = pd.read_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/filtered_valuation_df.csv')
# Add sector column to the DataFrame
filtered_valuation_df['sector'] = filtered_valuation_df['Tickers'].apply(get_sector)
filtered_valuation_df.head()

,marketCap,trailingPE,profitMargins,trailingEps,Tickers,Sharpe_ratios,sector
0,3.048763e+12,49.205574,0.55041,2.53,NVDA,3.295244,Technology
1,8.113363e+10,30.603773,0.42131,9.54,MAR,2.143484,Consumer Cyclical
2,1.455590e+11,55.550484,0.40294,2.08,ANET,2.474921,Technology
3,5.458747e+10,12.749406,0.39410,4.21,MPLX,2.439088,Energy
4,3.067929e+12,33.254635,0.35608,12.41,MSFT,2.195850,Technology


In [3]:
# possibly change group by  
print('\n----Filtering Df----')


top_stocks_on_profit = get_top_n_by_sector(filtered_valuation_df, filter_var='profitMargins', top_n=3)
top_stocks_on_sharpe = get_top_n_by_sector(filtered_valuation_df, filter_var='Sharpe_ratios', top_n=3)

print(f'Filtered based on: Profit Margins')
print(f"Sector Count: {len(top_stocks_on_profit['sector'].unique())}")
print(f"We have the following sector(s): \n{top_stocks_on_profit['sector'].unique()}")

print(f'\nFiltered based on: Sharpe Ratio')
print(f"Sector Count: {len(top_stocks_on_sharpe['sector'].unique())}")
print(f"We have the following sector(s): \n{top_stocks_on_sharpe['sector'].unique()}")


----Filtering Df----
Filtered based on: Profit Margins
Sector Count: 8
We have the following sector(s): 
['Communication Services' 'Consumer Cyclical' 'Energy'
 'Financial Services' 'Healthcare' 'Industrials' 'Technology' 'Utilities']

Filtered based on: Sharpe Ratio
Sector Count: 8
We have the following sector(s): 
['Communication Services' 'Consumer Cyclical' 'Energy'
 'Financial Services' 'Healthcare' 'Industrials' 'Technology' 'Utilities']


Diversifaction: Find negative correlated stocks

In [4]:
print('\n----Begining Diversifaction----')
profit_tickers_to_do_corr = top_stocks_on_profit['Tickers'].tolist()
top_profit_corr_pairs = get_corr_pairs_of_stocks(profit_tickers_to_do_corr)

sharpe_tickers_to_do_corr = top_stocks_on_sharpe['Tickers'].tolist()
top_sharpe_corr_pairs = get_corr_pairs_of_stocks(sharpe_tickers_to_do_corr)


[**********************56%**                     ]  9 of 16 completed


----Begining Diversifaction----


[*********************100%***********************]  16 of 16 completed
[*********************100%***********************]  16 of 16 completed


Seperate low corr pairs

In [5]:
corr_profit_pairs = top_profit_corr_pairs.index.to_list()
corr_sharpe_pairs = top_sharpe_corr_pairs.index.to_list()

mpt_stocks_for_sharpe = separate_corr_pairs(corr_sharpe_pairs, top_n=TOP_N_STOCKS)
mpt_stocks_for_profit = separate_corr_pairs(corr_profit_pairs, top_n=TOP_N_STOCKS)
print(f'Top Stocks based on Sharpe: {mpt_stocks_for_sharpe}')
print(f'Top Stocks based on Profit Margin: {mpt_stocks_for_profit}')

Top Stocks based on Sharpe: ['APP' 'PDD' 'MPLX' 'BKNG' 'META' 'FICO' 'COIN' 'LLY']
Top Stocks based on Profit Margin: ['MPLX' 'PDD' 'MAR' 'HLT' 'META' 'ANET' 'MSFT' 'UNP']


In [6]:
with open('utils/config.py', 'a') as f:
    f.write('\n# New Entry from portfolio_selection.ipynb')
    f.write(f'\nmpt_stocks_for_sharpe = {(mpt_stocks_for_sharpe).tolist()}\n')
    f.write(f'mpt_stocks_for_profit = {(mpt_stocks_for_profit).tolist()}\n')